<a href="https://colab.research.google.com/github/pradeep2c1/Machine-Translation-model/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import string
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

import re

In [2]:
#!pip install seaborn

In [3]:
#!nvidia-smi

In [4]:
tf.test.gpu_device_name()

''

# Data pre-processing

In [5]:
df = pd.read_csv(r'urdu_english.csv', encoding='utf-8')

In [6]:
#df['source'].value_counts()
df= df[:5000]

In [7]:
#df = df[df['source'] == 'ted']
df.head(10)

,english_sentence,urdu_sentence
0,what's the name of the movie,film ka kya naam hai
1,"hi, the rotten tomatoes score is great but the...","namaste, sada hua tomatoes score mahaan hai, l..."
2,do you think you will like the movie,kya aapako lagata hai ki aapako film pasand aa...
3,what kind of movie is it,yah kis tarah kee philm hai
4,when was the movie made?,film kab banee thee?
5,"wonder woman, i think i would enjoy this movie...","aashchary hai ki mahila, mujhe lagata hai ki m..."
6,whats the name of the movie,film ka kya naam hai
7,it is a action movie set in the dc comic world,yah deesee komik duniya mein sthaapit ek eksha...
8,who stars in the movie,: film mein kaun sitaare hain
9,the movie was made in 2015,movie 2015 mein banee thee


In [8]:
#df.drop(columns = ['source'], inplace = True)
df.shape

(5000, 2)

In [9]:
df.reset_index(inplace = True)
#df.drop(columns = ['index'], inplace = True)

In [10]:
pd.isnull(df).sum()

index               0
english_sentence    0
urdu_sentence       0
dtype: int64

In [11]:
df.drop_duplicates(inplace=True)
df.shape

(5000, 3)

In [12]:
df.head()

,index,english_sentence,urdu_sentence
0,0,what's the name of the movie,film ka kya naam hai
1,1,"hi, the rotten tomatoes score is great but the...","namaste, sada hua tomatoes score mahaan hai, l..."
2,2,do you think you will like the movie,kya aapako lagata hai ki aapako film pasand aa...
3,3,what kind of movie is it,yah kis tarah kee philm hai
4,4,when was the movie made?,film kab banee thee?


In [13]:
# Make all english letters lowercase
df['english_sentence'] = df['english_sentence'].apply(lambda x: x.lower())

# Remove the quotes
df['english_sentence'] = df['english_sentence'].apply(lambda x: re.sub("'", '', x))
df['urdu_sentence'] = df['urdu_sentence'].apply(lambda x: re.sub("'", '', x))

# Remove special characters
sp_char = set(string.punctuation)
df['english_sentence'] = df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in sp_char))
df['urdu_sentence'] = df['urdu_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in sp_char))

In [14]:
# Remove extra spaces
df['english_sentence']=df['english_sentence'].apply(lambda x: x.strip())
df['urdu_sentence']=df['urdu_sentence'].apply(lambda x: x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['urdu_sentence']=df['urdu_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [15]:
# Add START and END tokens to the beginning and end of the target sequence
df['english_sentence'] = df['english_sentence'].apply(lambda x: 'START_ ' + x + ' _END')

In [16]:
df.head()

,index,english_sentence,urdu_sentence
0,0,START_ whats the name of the movie _END,film ka kya naam hai
1,1,START_ hi the rotten tomatoes score is great b...,namaste sada hua tomatoes score mahaan hai lek...
2,2,START_ do you think you will like the movie _END,kya aapako lagata hai ki aapako film pasand aa...
3,3,START_ what kind of movie is it _END,yah kis tarah kee philm hai
4,4,START_ when was the movie made _END,film kab banee thee


In [17]:
df = df.sample(frac = 1).reset_index(drop = True)
df.head()

,index,english_sentence,urdu_sentence
0,2359,START_ what rotton tomato score did it get _END,isko rotten tomato score kitna mila
1,591,START_ i know _END,muje pata hai
2,2920,START_ are you still there _END,tum abhee bhi vahaan ho
3,982,START_ im not sure i liked how that was when m...,mujhe sure nahi hu mujhe yah pasand aaya ki ja...
4,1355,START_ i loved that hulk saved iron man when h...,muje acha lga ki hulk ne iron man ko bachaya g...


# Tokenizing the data

In [18]:
en = set()
for sentence in df['english_sentence']:
    for word in sentence.split():
        if word not in en:
            en.add(word)

hi = set()
for sentence in df['urdu_sentence']:
    for word in sentence.split():
        if word not in hi:
            hi.add(word)

In [19]:
print('Unique words in English are : ', len(en))
print('Unique words in urdu are : ', len(hi))

Unique words in English are :  5216
Unique words in urdu are :  7410


In [20]:
df['length_en'] = df['english_sentence'].apply(lambda x: len(x.split()))
df['length_ur'] = df['urdu_sentence'].apply(lambda x: len(x.split()))

df.head()

,index,english_sentence,urdu_sentence,length_en,length_ur
0,2359,START_ what rotton tomato score did it get _END,isko rotten tomato score kitna mila,9,6
1,591,START_ i know _END,muje pata hai,4,3
2,2920,START_ are you still there _END,tum abhee bhi vahaan ho,6,5
3,982,START_ im not sure i liked how that was when m...,mujhe sure nahi hu mujhe yah pasand aaya ki ja...,32,41
4,1355,START_ i loved that hulk saved iron man when h...,muje acha lga ki hulk ne iron man ko bachaya g...,17,17


In [21]:
print(df[df['length_en'] > 20].shape)
print(df[df['length_ur'] > 20].shape)

(747, 5)
(656, 5)


In [22]:
# df = df[df['length_en'] <= 20]
# df = df[df['length_ur'] <= 20]
# print(df.shape)

In [23]:
print("Maximum sequence length of inputs =", max(df['length_en']))
print("Maximum sequence length of outputs =", max(df['length_ur']))

Maximum sequence length of inputs = 249
Maximum sequence length of outputs = 273


In [24]:
max_encoder_seq_length = max(df['length_en'])
max_decoder_seq_length = max(df['length_ur'])

input_words = sorted(list(hi))
target_words = sorted(list(en))
num_encoder_tokens = len(hi)
num_decoder_tokens = len(en)
num_encoder_tokens, num_decoder_tokens

(7410, 5216)

In [25]:
num_encoder_tokens += 1
num_decoder_tokens += 1

In [26]:
input_token_id = dict([(word, i + 1) for i, word in enumerate(input_words)])
target_token_id = dict([(word, i + 1) for i, word in enumerate(target_words)])

rev_input_char_id = dict((i, word) for word, i in input_token_id.items())
rev_target_char_id = dict((i, word) for word, i in target_token_id.items())

In [27]:
df = shuffle(df)
df.head()

,index,english_sentence,urdu_sentence,length_en,length_ur
1896,3010,START_ yes very interesting _END,ha bahaut interesting,5,3
2313,1920,START_ ha ha critics said movie shows that his...,ha ha critics bole movie dikhati hai ki histor...,12,11
1291,4575,START_ he was probably really sad _END,woh shaayad waakayi mein sad tha,7,6
1807,4960,START_ harrison ford is legend _END,hairisan phord kinvadantee hai,6,4
3593,2895,START_ i really would recommend it myself it w...,tho mein jarur recommend karunga yeh ache hein...,25,24


# Splitting the data set

In [28]:
from sklearn.model_selection import train_test_split

# Splitting the data into 80% train and 20% test
x_train, x_test, y_train, y_test = train_test_split(df['urdu_sentence'],df['english_sentence'], test_size=0.2, random_state=42)

# print("Train set size:", len(x_train))
# print("Test set size:", len(x_test))
# Create a DataFrame to store x_train and y_train together
train_data = pd.DataFrame({'urdu_sentence': x_train, 'english_sentence': y_train})

# Sort the train_data DataFrame by urdu_sentence in descending order
sorted_train_df = train_data.sort_values(by='urdu_sentence', key=lambda x: x.str.len(), ascending=False)

# Retrieve sorted x_train and y_train
x_train = sorted_train_df['urdu_sentence']
y_train = sorted_train_df['english_sentence']

In [29]:
x_train = sorted(x_train, key=lambda x: len(x), reverse=True)

# Print the sorted x_train
for sentence in x_train:
    print(sentence,'\n')

batman ne intercepts kiya us drug shipment ko lekin rachel us evidence se falcone ko against kiya aur sergeant james gordon ko enlists kiya yek imandar cop tho us gowtham ko left kar de arrest karne keliye prison mein falcone dr jonathan crane se mila jo yek corrupt psychiatrist hein jo gotham ko drugs dene par help kiya aur use threatens bhi kiya us complicitiy ko reveal karne pe agar voh mentally unfit hoga tho use declare nahi karega crane ne tho us scarecrow mask pehena aur falcone ka dar ko sprays out kiya voh hallucinogen tha kaafi insane huva fir us arkham asylum ko transferred bhi kiya jab scarecrow ko investigating kar rahe the tab batman ne us hallucinogen ko exposed kiya aur incapacitated ko left kiya voh alfred ko save kiya aur usse fox ka antidote developed de deya jab rachel ko crane ka correptionko accusses kiya gaya tab crane ne reveal kiya ki usse drug se dar hein aur voh gowtham ka water supply tha voh rachel ko drugs addit kiya lekin batman ne usse save kara aur cran

In [30]:
def generate_batch(x = x_train, y = y_train, batch_size = 128):
    #print(x_train)
    while True:
        for j in range(0, len(x), batch_size):
            
            encoder_input_data = np.zeros((batch_size, max_encoder_seq_length), dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_decoder_seq_length), dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(x[j:j + batch_size], y[j:j + batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_id[word]   # encoder input seq
                for t, word in enumerate(target_text.split()):    
                    if t < len(target_text.split()) - 1:
                        decoder_input_data[i, t] = target_token_id[word]   # decoder input seq
                    if t > 0:
                        # does not include the START_ token
                        decoder_target_data[i, t - 1, target_token_id[word]] = 1.
            
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Making the Model

In [31]:
latent_dim = 256

In [32]:
encoder_inputs = Input(shape=(None,))  # Create the input tensor
encoder_embedding =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)

encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
# We discard encoder_outputs and only keep the states
encoder_states = [state_h, state_c]

In [33]:
# Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
 
# We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn encoder_input_data & decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [34]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            1897216   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            1335552   ['input_2[0][0]']             
                                                                                              

In [36]:
train_samples = len(x_train)
val_samples = len(x_test)
batch_size = 128
epochs = 100

In [37]:
model.fit(
    generate_batch(x_train, y_train, batch_size = batch_size),
    steps_per_epoch = train_samples//batch_size,
    epochs = epochs,
    validation_data = generate_batch(x_test, y_test, batch_size = batch_size),
    validation_steps = val_samples//batch_size)

IndexError: index 249 is out of bounds for axis 1 with size 249

In [ ]:
# model_json = model.to_json()
# with open("eng_urdu_model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("eng_urdu_weights.h5")
# print("Saved model to disk")

# loading the model architecture and asigning the weights
json_file = open('model_2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()


In [ ]:
#model.save('eng_urdu_weights.h5')

In [ ]:
model_loaded = model_from_json(loaded_model_json)
# load weights into new model
model_loaded.load_weights("model_weight_5.h5")

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_inputs = model_loaded.input[0] #Trained encoder input layer
encoder_outputs, inf_state_h, inf_state_c = model_loaded.layers[4].output # retoring the encoder lstm output and states
encoder_states = [inf_state_h,inf_state_c]
encoder_model = Model(encoder_inputs, encoder_states)

#inference decoder
# The following tensor will store the state of the previous timestep in the "starting the encoder final time step"
decoder_state_h_input = Input(shape=(latent_dim,)) #becase during training we have set the lstm unit to be of 50
decoder_state_c_input = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_h_input,decoder_state_c_input]

# # inference decoder input
decoder_input_inf = model_loaded.input[1] #Trained decoder input layer
# decoder_input_inf._name='decoder_input'
decoder_emb_inf = model_loaded.layers[3](decoder_input_inf)
decoder_lstm_inf = model_loaded.layers[5]
decoder_output_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm_inf(decoder_emb_inf, initial_state =decoder_state_input)
decoder_state_inf = [decoder_state_h_inf,decoder_state_c_inf]
#inference dense layer
dense_inf = model_loaded.layers[6]
decoder_output_final = dense_inf(decoder_output_inf)# A dense softmax layer to generate prob dist. over the target vocabulary

decoder_model = Model([decoder_input_inf]+decoder_state_input,[decoder_output_final]+decoder_state_inf)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_id['START_']

    # Sampling loop for a batch of sequences
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = rev_target_char_id[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
train_gen = generate_batch(x_train, y_train, batch_size = 1)
k=-1

In [ ]:
urdu_sentence= input('Enter urdu sentence: ')
tokenizer_input= Tokenizer()
input_seq= tokenizer_input.texts_to_sequences([urdu_sentence])
pad_seq= pad_sequences(input_seq)
predicted_target = decode_sequence(pad_seq)
print("predicted Translate:",predicted_target[:-4])

In [ ]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual urdu Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted urdu Translation:', decoded_sentence[:-4])

In [ ]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual urdu Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted urdu Translation:', decoded_sentence[:-4])

In [ ]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual urdu Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted urdu Translation:', decoded_sentence[:-4])

In [ ]:
k += 1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', x_train[k:k + 1].values[0])
print('Actual urdu Translation:', y_train[k:k + 1].values[0][6:-4])
print('Predicted urdu Translation:', decoded_sentence[:-4])

## New

In [ ]:
from tensorflow.keras.models import load_model

# Load the weights into your model
#model.load_weights('eng_urdu_weights.h5')

# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate probability over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_id['START_']

    # Sampling loop for a batch of sequences
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = rev_target_char_id[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or len(decoded_sentence) > 50):
            break

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

train_gen = generate_batch(x_train, y_train, batch_size=1)
k=-1

k += 1
# (input_seq, actual_output), _ = next(train_gen)
# print(type(input_seq))
# decoded_sentence = decode_sequence(input_seq)
# print('Input English sentence:', x_train[k:k + 1].values[0])
# print('Actual urdu Translation:', y_train[k:k + 1].values[0][6:-4])
# print('Predicted urdu Translation:', decoded_sentence[:-4])

# urdu_sentence= input('Enter urdu sentence: ')
# tokenizer_input= Tokenizer()
# input_seq= tokenizer_input.texts_to_sequences([urdu_sentence])
# pad_seq= pad_sequences(input_seq)
# predicted_target = decode_sequence(pad_seq)
# print("predicted Translate:",predicted_target[:-4])